In [16]:

import functools
from math import sqrt

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import scipy.stats as sct
import seaborn as sns
from sklearn.datasets import load_digits, fetch_20newsgroups
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import  CountVectorizer, TfidfTransformer, TfidfVectorizer

from sklearn.impute import SimpleImputer
from sklearn.pipeline import  Pipeline
from sklearn.preprocessing import OneHotEncoder,Binarizer,KBinsDiscretizer, MinMaxScaler, StandardScaler, PolynomialFeatures


In [17]:
%matplotlib inline
from IPython.core.pylabtools import  figsize
figsize(12,12)
sns.set()


#### One Hot encoding



In [18]:
rows = 100
height = np.random.normal(loc=1.70, scale = 0.2, size = rows).round(3)
score = np.random.normal(loc=7, scale = 1, size = rows).round(2)
courses = ['Math','Physics','Biology']
course = np.random.choice(courses, size = rows)

data = pd.DataFrame({
        'Height': height,
        'Score': score ,
        'Course': course

})
data.head()

,Height,Score,Course
0,1.350,7.64,Biology
1,1.623,7.78,Biology
2,1.799,8.50,Biology
3,1.669,6.46,Math
4,1.880,7.59,Physics


###### Pandas

In [19]:
data_encoded = pd.get_dummies(data=data, prefix='Course')

###### Sklearn

In [20]:
one_hot_encoder = OneHotEncoder(sparse=False, dtype=np.int)
course_encoded = one_hot_encoder.fit_transform(data[['Course']])
course_encoded[0:10]


/tmp/ipykernel_7095/731080035.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  one_hot_encoder = OneHotEncoder(sparse=False, dtype=np.int)


array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

###### Binarização


In [21]:
tail = (data.Height> 1.80)
tail[:10]

0    False
1    False
2    False
3    False
4     True
5     True
6    False
7    False
8    False
9     True
Name: Height, dtype: bool

In [22]:
binarizer = Binarizer(threshold=1.80).fit(data_encoded[['Height']])
height_binary= binarizer.transform(data_encoded[['Height']])
height_binary[:10]


array([[0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.]])

In [23]:
height_bool = pd.DataFrame(height_binary.flatten().astype(bool), columns = ['Tail'])
height_bool.head(10)

,Tail
0,False
1,False
2,False
3,False
4,True
5,True
6,False
7,False
8,False
9,True


In [24]:
data = pd.concat([data_encoded, height_bool], axis=1)
data.head(10)

,Height,Score,Course_Biology,Course_Math,Course_Physics,Tail
0,1.350,7.64,1,0,0,False
1,1.623,7.78,1,0,0,False
2,1.799,8.50,1,0,0,False
3,1.669,6.46,0,1,0,False
4,1.880,7.59,0,0,1,True
5,1.896,8.21,1,0,0,True
6,1.766,5.24,0,1,0,False
7,1.567,6.70,0,0,1,False
8,1.641,7.31,0,0,1,False
9,2.093,7.45,0,0,1,True


,Height,Score,Course_Biology,Course_Math,Course_Physics,Tail
0,1.350,7.64,1,0,0,False
1,1.623,7.78,1,0,0,False
2,1.799,8.50,1,0,0,False
3,1.669,6.46,0,1,0,False
4,1.880,7.59,0,0,1,True
